In [8]:
%pylab inline
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

import librosa
import IPython.display as ipd

Populating the interactive namespace from numpy and matplotlib


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
class VADDNN(nn.Module):
    def __init__(self, input_size=40, output_size=496, dropout=0.5):
        super(VADDNN, self).__init__()
        self.linear_1 = nn.Linear(input_size, 256)
        self.linear_2 = nn.Linear(256, 256)
        self.linear_3 = nn.Linear(256, 256)
        self.linear_4 = nn.Linear(256, 256)
        self.linear_5 = nn.Linear(256, output_size)
        self.dropout_1 = nn.Dropout(p=dropout)
        self.dropout_2 = nn.Dropout(p=dropout)
            
    def forward(self, x, out_keys):
        out = {}
        out['h1'] = nn.functional.relu(self.linear_1(x))
        out['h2'] = nn.functional.relu(self.linear_2(out['h1']))
        out['h3'] = nn.functional.relu(self.linear_3(self.dropout_1(out['h2'])))
        out['h4'] = nn.functional.relu(self.linear_3(self.dropout_2(out['h3'])))
        out['out'] = nn.functional.relu(self.linear_5(out['h4']))
        return [out[key] for key in out_keys]

In [74]:
"""audio_folder = '/home/ubuntu/vad-experiments/dataset/LibriSpeech/train-clean-100/'

X = []
Y = []

speakers = os.listdir(audio_folder)
speaker2id = {}
flac_files = []
for speaker_id, speaker in enumerate(speakers):
    speaker2id[speaker] = speaker_id
    
for root, dirs, files in os.walk("/home/ubuntu/vad-experiments/dataset/LibriSpeech/train-clean-100"):
    for file in files:
        if file.endswith(".flac"):
             flac_files.append(os.path.join(root, file))
print(len(flac_files))
for file_num, audio_file in enumerate(flac_files):
    speaker = os.path.basename(os.path.dirname(os.path.dirname(audio_file)))
    y, sr = librosa.load(audio_file)
    num_samples = y.shape[0]
    int_seconds = int(num_samples/sr)
    y = y[:int_seconds * sr]
    samples = np.array(np.split(y, int_seconds))
    mfccs = []
    for i in range(len(samples)):
        X.append(np.mean(librosa.feature.mfcc(y=samples[i], sr=sr, n_mfcc=40).T,axis=0))
    Y += [speaker2id[speaker]] * len(samples)
    print(file_num)
newX = np.array(X)
newY = np.array(Y).reshape(-1, 1)
np.save('Xvals.np', newX)
np.save('Yvals.np', newY)
"""

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-74-55a14ba5d295>, line 32)

In [82]:
X = np.load('Xvals.np.npy')
Y = np.load('Yvals.np.npy')
print(X.shape, X.dtype)
print(Y.shape, Y.dtype)

(347745, 40) float32
(347745, 1) int64


dtype('float32')